<a href="https://colab.research.google.com/github/abhimshnew/Pneumonia_classification/blob/master/Pneumonia_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TODO**

1.   Get data from the kaggle
2.   preprocess the data from the folders
3.   Generate the data set
4.   reize each images to 299×299
5.   get the inceptionv3 model removing the last layer
6.   Add the sutable layer at the output to detect pneumonia or not pneumonia
7.   Get the precision and accuracy score
8.   Try to train some layers in Incepionv3 and see if the accuracy/precision score gets better






In [0]:
import os
import glob
import collections
from keras.models import Sequential
from keras.applications.inception_v3 import InceptionV3


import pandas as pd
import cv2
import numpy as np

In [4]:
#load the kaggle attenticaion

from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abhimshm","key":"852887340af85c7779cafbf907659999"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

 98% 1.13G/1.15G [00:06<00:00, 178MB/s]
100% 1.15G/1.15G [00:07<00:00, 175MB/s]


In [0]:
# !mkdir chest_xray
# !unzip /content/chest-xray-pneumonia.zip -d /content/chest_xray
# !unzip /content/chest_xray/chest_xray.zip -d /content/chest_xray/chest_xray
# Make sure to delet all the unnecessary zip folder to free up the memory

In [0]:
# define the directories path for train, test and validation
# here we observe that all the directories have same sub directories namely NORMAL and PNEUMONIA
train_path = '/content/chest_xray/chest_xray/chest_xray/train'
test_path = '/content/chest_xray/chest_xray/chest_xray/train'
valid_path = '/content/chest_xray/chest_xray/chest_xray/train'
dict_results = {'NORMAL':0, 'PNEUMONIA':1}

In [0]:
# Traverse through each image and develop the dataset needed with image as X and 
# Y as classifaction.i.e. Normal as 0 and pneumonia as 1
# Below step will be performaed for all data set hence putting it as a function
from google.colab.patches import cv2_imshow
def generate_data_set(path = None):
    X = []
    Y = []
    assert path is not None, "Please provide the path"
    for each_class in dict_results.keys():
        for each_image in glob.glob(os.path.join(path, each_class)+ "/*.jpeg"):
            img = cv2.imread(each_image)
            X.append(cv2.resize(img, (299,299), cv2.INTER_AREA))
            Y.append(dict_results[each_class])
    return X, Y

X, Y = generate_data_set(train_path)

The Image resolution is more than 299x299 as required by the inception3 model input layers, Hence need to shrink the image.


In [38]:
X[0].shape

(299, 299, 3)

In [42]:
Y = np.array(Y)
collections.Counter(Y)

Counter({0: 1341, 1: 3875})

In [46]:
base_model = InceptionV3(include_top=False, input_shape=(299, 299, 3),
                         pooling='avg')
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 149, 149, 32) 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 149, 149, 32) 0           batch_normalization_95[0][0]     
_______________________________________________________________________________________